# Synthetix V3


In [1]:
import os
import time
from synthetix import Synthetix
from synthetix.utils import wei_to_ether, ether_to_wei, format_wei, format_ether
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
snx = Synthetix(
    provider_rpc=os.getenv("LOCAL_RPC"),
    is_fork=True,
    cannon_config={
        "package": "synthetix-omnibus",
        "version": "latest",
        "preset": "octopus",
    },
)

2024-07-24 14:50:39,291 - INFO - Using RPC signer: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
2024-07-24 14:50:39,295 - INFO - Setting network_id from RPC chain_id: 421614
2024-07-24 14:50:39,676 - INFO - Loading cannon contracts for synthetix-omnibus:latest@octopus
2024-07-24 14:50:44,367 - WARNING - Failed to fetch core accounts: HTTPConnectionPool(host='localhost', port=8545): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x136b014e0>: Failed to establish a new connection: [Errno 61] Connection refused'))


### Setup


In [ ]:
# get some contracts
usdx = snx.spot.markets_by_id[0]["contract"]
weth = snx.contracts["WETH"]["contract"]

# USDC token
usdc_package = snx.contracts["usdc_mock_collateral"]["MintableToken"]
usdc = snx.web3.eth.contract(address=usdc_package["address"], abi=usdc_package["abi"])

# BTC token
btc_package = snx.contracts["btc_mock_collateral"]["MintableToken"]
btc = snx.web3.eth.contract(address=btc_package["address"], abi=btc_package["abi"])

In [ ]:
def status():
    """A function printing some useful information about the connected account"""
    eth_balance = snx.get_eth_balance()
    susd_balance = snx.spot.get_balance(market_name="sUSD")

    usdc_balance = usdc.functions.balanceOf(snx.address).call()
    usdc_balance = format_wei(usdc_balance, 6)

    btc_balance = btc.functions.balanceOf(snx.address).call()
    btc_balance = format_wei(btc_balance, 18)

    print(
        f"""
    Perps accounts: {snx.perps.account_ids}
    Core accounts: {snx.core.account_ids}
        
    ETH: {eth_balance['eth']}
    WETH ({weth.address}): {eth_balance['weth']}
    sUSD: {susd_balance}
    USDC ({usdc.address}): {usdc_balance}
    BTC ({btc.address}): {btc_balance}
    """
    )


status()

In [ ]:
def find_contract(contracts, signature=None, address=None):
    def search_recursive(item, path=[]):
        if isinstance(item, dict):
            if "contract" in item and "abi" in item and "address" in item:
                # This is a contract
                contract = item["contract"]

                # check the address
                if address and item["address"] == address:
                    return ".".join(path), contract

                try:
                    function = contract.get_function_by_signature(signature)
                    if function:
                        return ".".join(path), contract
                except Exception as e:
                    # If get_function_by_signature raises an exception, we'll just continue
                    pass
            else:
                # This might be a package or a nested structure
                for key, value in item.items():
                    result = search_recursive(value, path + [key])
                    if result:
                        return result
        return None

    result = search_recursive(contracts)
    return result if result else (None, None)

In [ ]:
find_contract(snx.contracts, signature="0xf65d3e18")

In [ ]:
contract_name, contract = find_contract(
    snx.contracts, address="0x1FBbaFB9232247D37aF889210c75BbF29303b833"
)

In [ ]:
contract_name

In [ ]:
contract = snx.contracts["perps_gas_oracle_node"]["ArbGasPriceOracle"]["contract"]

In [ ]:
arbgas = snx.contracts["GasOracle"]["contract"]
arbgas

In [ ]:
arbgas.functions.getPricesInWei().call()